In [1]:
import pandas as pd
import glob
import os

In [2]:
KENPOM_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/KenPom"
KAGGLE_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/MDataFiles_Stage2"
OUTPUT_DATA_DIR = "/Users/jameswhedbee/jupyter_notebooks/DATA/MarchMadness/output"

In [3]:
KENPOM_COLS = ["Season", "TeamName", "AdjOE", "AdjDE", "AdjTempo"]

In [4]:
all_files = glob.glob(os.path.join(KENPOM_DATA_DIR, "*.csv"))

data = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    data.append(df)

kenpom_df = pd.concat(data, axis=0, ignore_index=True)[KENPOM_COLS]
kenpom_df["LowerTeamName"] = kenpom_df["TeamName"].str.lower()
kenpom_df

,Season,TeamName,AdjOE,AdjDE,AdjTempo,LowerTeamName
0,2013,Air Force,112.8840,104.4400,62.5525,air force
1,2013,Akron,107.8010,94.0583,64.6243,akron
2,2013,Alabama,104.1100,93.3046,59.7128,alabama
3,2013,Alabama A&M,90.5796,111.4980,64.2054,alabama a&m
4,2013,Alabama St.,92.2111,112.0520,64.0774,alabama st.
...,...,...,...,...,...,...
6182,2004,Wright St.,103.6850,104.8470,63.4202,wright st.
6183,2004,Wyoming,104.4840,99.8136,68.2696,wyoming
6184,2004,Xavier,110.7610,92.9239,64.1432,xavier
6185,2004,Yale,99.3150,103.3250,63.1637,yale


In [5]:
teams_df = pd.read_csv(f"{KAGGLE_DATA_DIR}/MTeams.csv")
spellings_df = pd.read_csv(f"{KAGGLE_DATA_DIR}/MTeamSpellings.csv")
spellings_df[870:899]

,TeamNameSpelling,TeamID
870,southern illinois,1356
871,southern illinois-edwardsville,1188
872,southern methodist,1374
873,southern miss,1379
874,southern miss.,1379
875,southern mississippi,1379
876,southern u,1380
877,southern u.,1380
878,southern un,1380
879,southern univ,1380


In [6]:
merged_df = kenpom_df.merge(spellings_df, left_on="LowerTeamName", right_on="TeamNameSpelling")[KENPOM_COLS + ["TeamID"]]
merged_df

,Season,TeamName,AdjOE,AdjDE,AdjTempo,TeamID
0,2013,Air Force,112.8840,104.4400,62.5525,1102
1,2011,Air Force,105.7170,101.1380,61.2714,1102
2,2003,Air Force,105.3400,103.7220,52.9909,1102
3,2019,Air Force,100.4310,107.2750,66.4070,1102
4,2007,Air Force,116.2860,97.0562,56.6656,1102
...,...,...,...,...,...,...
5912,2017,Fort Wayne,109.5300,108.0540,71.2523,1236
5913,2018,Fort Wayne,107.6160,106.4470,72.5983,1236
5914,2021,Bellarmine,107.4090,108.6840,64.6109,1468
5915,2021,Merrimack,90.8608,101.7180,66.7842,1467


In [7]:
missing_teams = set(kenpom_df["TeamName"]) - set(merged_df["TeamName"])
missing_teams

{'Arkansas Little Rock',
 'Arkansas Pine Bluff',
 'Bethune Cookman',
 'Cal St. Bakersfield',
 'Dixie St.',
 'Illinois Chicago',
 'LIU',
 'Louisiana Lafayette',
 'Louisiana Monroe',
 'Maryland Eastern Shore',
 'Mississippi Valley St.',
 'Southeast Missouri St.',
 'Southwest Missouri St.',
 'Southwest Texas St.',
 'St. Francis NY',
 'St. Francis PA',
 'Tarleton St.',
 'Tennessee Martin',
 'Texas A&M Corpus Chris',
 'Texas Pan American',
 'UT Rio Grande Valley',
 'Winston Salem St.'}

In [8]:
missing_df = kenpom_df[kenpom_df["TeamName"].isin(missing_teams)]
missing_df["LowerTeamName"] = missing_df["LowerTeamName"].str.replace(" ", "-").str.replace("\.", "").str.replace("-st", "-state")
dash_merge_df = missing_df.merge(spellings_df, left_on="LowerTeamName", right_on="TeamNameSpelling")[KENPOM_COLS + ["TeamID"]]
merged_df = pd.concat([merged_df, dash_merge_df])
missing_teams = set(kenpom_df["TeamName"]) - set(merged_df["TeamName"])
missing_teams

<ipython-input-8-84477e04aab3>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_df["LowerTeamName"] = missing_df["LowerTeamName"].str.replace(" ", "-").str.replace("\.", "").str.replace("-st", "-state")
<ipython-input-8-84477e04aab3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df["LowerTeamName"] = missing_df["LowerTeamName"].str.replace(" ", "-").str.replace("\.", "").str.replace("-st", "-state")


{'Dixie St.',
 'LIU',
 'Southwest Missouri St.',
 'Southwest Texas St.',
 'Tarleton St.',
 'Texas A&M Corpus Chris',
 'UT Rio Grande Valley'}

In [9]:
missing_df[missing_df["TeamName"].isin(missing_teams)]

,Season,TeamName,AdjOE,AdjDE,AdjTempo,LowerTeamName
287,2013,Texas A&M Corpus Chris,96.0484,110.9970,60.1111,texas-a&m-corpus-chris
632,2011,Texas A&M Corpus Chris,92.2646,108.2170,64.8912,texas-a&m-corpus-chris
773,2003,Southwest Missouri St.,100.6670,92.4722,63.5846,southwest-missouri-state
884,2003,Southwest Texas St.,101.7770,104.5450,67.5136,southwest-texas-state
942,2003,Texas A&M Corpus Chris,99.2130,108.4200,72.8476,texas-a&m-corpus-chris
1311,2019,Texas A&M Corpus Chris,94.4776,104.4210,64.7448,texas-a&m-corpus-chris
1338,2019,UT Rio Grande Valley,99.2620,99.6347,70.0964,ut-rio-grande-valley
1464,2007,Texas A&M Corpus Chris,114.2010,106.4770,67.5189,texas-a&m-corpus-chris
1998,2015,Texas A&M Corpus Chris,99.6450,104.9700,62.1247,texas-a&m-corpus-chris
2349,2017,Texas A&M Corpus Chris,102.9550,106.2110,67.8840,texas-a&m-corpus-chris


In [10]:
merged_df.to_csv(f"{OUTPUT_DATA_DIR}/kenpom.csv", index=False)